# Initialize

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.9.1` // Not required since almond 0.7.0 (will be automatically added when importing spark)

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

//jupyter notebook way to create spark session
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                               // Not required since almond 0.7.0 (will be automatically added when importing spark)


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

//jupyter notebook way to create spark session

spark: SparkSession = org.apache.spark.sql.SparkSession@15b68a0

# Get spark context

In [2]:
val sc = spark.sparkContext

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@114a4f

# run some test

In [3]:
val rddwork = sc.textFile("/home/pi/sparktest/testdata.csv")

rddwork: org.apache.spark.rdd.RDD[String] = /home/pi/sparktest/testdata.csv MapPartitionsRDD[1] at textFile at cmd2.sc:1

# group all phone and create RDD

In [4]:
def splitFields (line:String):Tuple2[String,String]={
    val fields = line.split("\t")
    return (fields(0),fields(1) + "\t" + fields(2) + "\t" + fields(3)
           + "\t" + fields(4)
           + "\t" + fields(5)
           + "\t" + fields(6)
           + "\t" + fields(7)
           + "\t" + fields(8)
           + "\t" + fields(9)
           + "\t" + fields(10)
           + "\t" + fields(11))
}

//print(splitFields("1\t5\t3344\tnull"))
val header = rddwork.first()
val rddworkkv = rddwork.filter(_!=header).map(t=>splitFields(t))

first at cmd3.sc:15

1 / 1

defined function splitFields
header: String = "1\t185\t\"US\"\t\"100002\"\t\"Web Identity\"\t\"2018-09-21-00.28.50.855000\"\t111\t100001\t3344\t\"null\"\t100001\t\"11\""
rddworkkv: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[3] at map at cmd3.sc:16

# group by and show result

In [5]:
val testrdd = rddworkkv.groupByKey()
testrdd.take(3)

map at cmd3.sc:16

2 / 2

take at cmd4.sc:2

1 / 1

testrdd: org.apache.spark.rdd.RDD[(String, Iterable[String])] = ShuffledRDD[4] at groupByKey at cmd4.sc:1
res4_1: Array[(String, Iterable[String])] = Array(
  (
    "4",
    CompactBuffer(
      "185\t\"US\"\t\"100002\"\t\"Web Identity\"\t\"2018-04-21-00.28.50.855000\"\t113\t100001\t2455\t2455\t100001\t\"14\"",
      "185\t\"US\"\t\"100002\"\t\"Web Identity\"\t\"2018-04-21-00.28.50.855000\"\t113\t5\t5134323\t5134323\t100001\t\"14\"",
      "185\t\"US\"\t\"100002\"\t\"Web Identity\"\t\"2018-04-21-00.28.50.855000\"\t113\t100002\t+1-5134323\t5134323\t100001\t\"14\""
    )
  ),
  (
    "6",
    CompactBuffer(
      "185\t\"US\"\t\"100002\"\t\"Web Identity\"\t\"2018-05-21-00.28.50.855000\"\t113\t2\t12\t12\t100001\t\"16\"",
      "185\t\"US\"\t\"100002\"\t\"Web Identity\"\t\"2018-05-21-00.28.50.855000\"\t113\t5\t13\t\"null\"\t100001\t\"16\""
    )
  ),
  (
    "2",
    CompactBuffer(
      "185\t\"US\"\t\"100002\"\t\"Web Identity\"\t\"2018-01-21-00.28.50.855000\"\t113\t100001\t+(1)34434\t\"n

# filter to find cell phone not equals to others

In [6]:
import scala.collection.mutable.ListBuffer
def isCellDifferent(arg:Iterable[String]):Boolean={
    var result = true;
    var cell = "null"
    var cellSt = "null"
    
    var work = "null"
    var workSt = "null"
    var alt = "null"
    var altSt = "null"
    
    var otherPhoneList = ListBuffer[(String,String)]()
    
    for (str <- arg){
        val fields = str.split("\t")
        if(fields(0) == "5"){
            cell = fields(1)
            cellSt = fields(2)
        }
        else{
            otherPhoneList += ((fields(1),fields(2)))
        }
    }
    
    if(cell != "null" || cellSt != "null"){
        result = true
        for(otherPhone <- otherPhoneList){
            if(cellSt != "null" && otherPhone._2 != "null"){
                if(cellSt == otherPhone._2)
                    result = false
            }else{
                if(cell == otherPhone._1)
                    result = false
            }
        }
    }
    else{
        //no cell phone, return false
        result = false
    }
    
    return result
}
//test data:
var testList = ListBuffer[String, ]()
testList += (
    ("4",
              (
      "185\t\"US\"\t\"100002\"\t\"Web Identity\"\t\"2018-04-21-00.28.50.855000\"\t113\t100001\t2455\t2455\t100001\t\"14\"",
      "185\t\"US\"\t\"100002\"\t\"Web Identity\"\t\"2018-04-21-00.28.50.855000\"\t113\t5\t5134323\t5134323\t100001\t\"14\"",
      "185\t\"US\"\t\"100002\"\t\"Web Identity\"\t\"2018-04-21-00.28.50.855000\"\t113\t100002\t+1-5134323\t5134323\t100001\t\"14\""
                )
    )
            )
//var testlist = ListBuffer(("1", ("2\t3344\t3344", "5\t3344\tnull", "100001\t3344\tnull"))
//                   ,("2", ("5\t134434\t134434", "100001\t134434\tnull"))
//                  ,("3", ("5\t5388\t5388", "100001\t8988\t8988")))

//testrdd.filter(t=>isCellDifferent(t._2)).collect()


cmd6.sc:49: type mismatch;
 found   : (String, (String, String, String))
 required: Nothing
    ("4",
    ^Compilation Failed

: 